In [1]:
from langchain_core.documents import Document
documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
        metadata={"source": "fish-pets-doc"},
    ),
    Document(
        page_content="Parrots are intelligent birds capable of mimicking human speech.",
        metadata={"source": "bird-pets-doc"},
    ),
    Document(
        page_content="Rabbits are social animals that need plenty of space to hop around.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

In [2]:
documents

[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care.'),
 Document(metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.')]

In [14]:
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_groq import ChatGroq
groq_api_key = os.getenv("GROA_API_KEY")
groq_api_key
os.environ["HF_TOKEN"]=os.getenv("HF_TOKEN")
model = ChatGroq(model="llama-3.3-70b-versatile",groq_api_key=groq_api_key)
model 

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x10b337760>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x10b35cc10>, model_name='llama-3.3-70b-versatile', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [16]:
# using huggingface embedding 
from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [17]:
# vector store
from langchain_chroma import Chroma
vectorstore=Chroma.from_documents(documents,embedding=embeddings)

In [18]:
vectorstore.similarity_search("human")

[Document(id='28391338-c07b-432b-aaf4-3a7a8438296a', metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.'),
 Document(id='2f83db4e-ba5b-4634-8676-fb16e8d68a88', metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.'),
 Document(id='caf829f2-6859-4857-b0c2-782e754037b6', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(id='d750b8f1-ecde-4caa-9fd4-2721b1c99d98', metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care.')]

In [19]:
## Async query
await vectorstore.asimilarity_search("dog")

[Document(id='caf829f2-6859-4857-b0c2-782e754037b6', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(id='f4c8b3ae-be86-4aee-aad1-cbec3384355b', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(id='d750b8f1-ecde-4caa-9fd4-2721b1c99d98', metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care.'),
 Document(id='2f83db4e-ba5b-4634-8676-fb16e8d68a88', metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.')]

In [20]:
vectorstore.similarity_search_with_score("cat")

[(Document(id='f4c8b3ae-be86-4aee-aad1-cbec3384355b', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
  0.9351049661636353),
 (Document(id='caf829f2-6859-4857-b0c2-782e754037b6', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
  1.5740902423858643),
 (Document(id='2f83db4e-ba5b-4634-8676-fb16e8d68a88', metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.'),
  1.595691442489624),
 (Document(id='28391338-c07b-432b-aaf4-3a7a8438296a', metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.'),
  1.6657934188842773)]

In [21]:
#without subclassing Retriever

from langchain_core.runnables import RunnableLambda
retriever1=RunnableLambda(vectorstore.similarity_search).bind(k=1)
retriever1.batch(["cat","dog"])

[[Document(id='f4c8b3ae-be86-4aee-aad1-cbec3384355b', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')],
 [Document(id='caf829f2-6859-4857-b0c2-782e754037b6', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]]

In [23]:
# retriver 

retriever=vectorstore.as_retriever(search_type="similarity",
    search_kwargs={"k":1})
retriever.batch(["human","dog"])

[[Document(id='28391338-c07b-432b-aaf4-3a7a8438296a', metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.')],
 [Document(id='caf829f2-6859-4857-b0c2-782e754037b6', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]]

In [24]:
#Rag
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message = """
Answer this question using the provided context only.

{question}

Context:
{context}
"""

prompt = ChatPromptTemplate.from_messages([("human",message)])
# defining the context and question 
rag_chain={"context":retriever,"question":RunnablePassthrough()}|prompt|model

response=rag_chain.invoke("tell me about dogs")
print(response.content)

Dogs are great companions, known for their loyalty and friendliness.
